<a href="https://colab.research.google.com/github/stevecoffee/dasc5300/blob/main/Project2/DASC_5300_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-igraph

     |████████████████████████████████| 3.2 MB 7.4 MB/s 


In [2]:
import pandas as pd
from igraph import *    # https://igraph.org/python/tutorial/0.9.6/tutorial.html

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

Grabbed additional data from https://openflights.org/data.html:
- airports.dat
- airports-extended.dat
- routes.dat
- countries.dat


In [133]:
df_openflight_airports = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/OpenFlights/airports.dat',
    names=['airport-id', 'name', 'city', 'country', 'iata', 'icao', 'latitude', 'longitude', 'altitude', 'timezone', 'DST', 'database-timezone', 'type', 'source'],
    skipinitialspace='true', header=0)
# df_openflight_airports.reset_index(level=0, inplace=True)  # add row number as a column to use as vertex ID with graph library
print(df_openflight_airports)
print(df_openflight_airports.dtypes)
df_openflight_airports['airport-id'] = df_openflight_airports['airport-id'].astype(str)
print(df_openflight_airports.dtypes)
print(df_openflight_airports['airport-id'])

# create airline to vertex ID dictionary

print(airportDict)

      airport-id                                         name          city           country iata  icao   latitude   longitude  altitude timezone DST     database-timezone     type       source
0              2                               Madang Airport        Madang  Papua New Guinea  MAG  AYMD  -5.207080  145.789001        20       10   U  Pacific/Port_Moresby  airport  OurAirports
1              3                 Mount Hagen Kagamuga Airport   Mount Hagen  Papua New Guinea  HGU  AYMH  -5.826790  144.296005      5388       10   U  Pacific/Port_Moresby  airport  OurAirports
2              4                               Nadzab Airport        Nadzab  Papua New Guinea  LAE  AYNZ  -6.569803  146.725977       239       10   U  Pacific/Port_Moresby  airport  OurAirports
3              5  Port Moresby Jacksons International Airport  Port Moresby  Papua New Guinea  POM  AYPY  -9.443380  147.220001       146       10   U  Pacific/Port_Moresby  airport  OurAirports
4              6         

In [7]:
df_openflight_countries = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/OpenFlights/countries.dat',
    names=['name', 'iso-code', 'dafif-code'],
    skipinitialspace='true', header=0)
print(df_openflight_countries)

                     name iso-code dafif-code
0                   Aruba       AW         AA
1     Antigua and Barbuda       AG         AC
2    United Arab Emirates       AE         AE
3             Afghanistan       AF         AF
4                 Algeria       DZ         AG
..                    ...      ...        ...
255                 Samoa       WS         WS
256              Eswatini       SZ         WZ
257                 Yemen       YE         YM
258                Zambia       ZM         ZA
259              Zimbabwe       ZW         ZI

[260 rows x 3 columns]


In [10]:
df_openflight_routes = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/OpenFlights/routes.dat',
    names=['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment'],
    skipinitialspace='true', header=0)
# print(df_routes)

# remove rows with missing data
# todo: can this be limited by excluding columns first?
no_missing_data = (df_openflight_routes != '\\N').all(axis='columns')
df_openflight_routes = df_openflight_routes[no_missing_data]

# remove rows with invalid source airports
# todo: can this be limited by patching from other sources?
valid_source = df_openflight_routes['source-airport-id'].isin(df_openflight_airports['airport-id'])
df_openflight_routes = df_openflight_routes[valid_source]

# remove rows with invalid dest airports
# todo: can this be limited by patching from other sources?
valid_dest = df_openflight_routes['dest-airport-id'].isin(df_openflight_airports['airport-id'])
df_openflight_routes = df_openflight_routes[valid_dest]

print(df_openflight_routes)

# reorder and fix datatypes to prepare for graph population
df_openflight_routes = df_openflight_routes[['source-airport-id', 'dest-airport-id', 'airline-code', 'airline-id', 'source-airport-code', 'dest-airport-code', 'codeshare', 'stops', 'equipment']]
df_openflight_routes['source-airport-id'] = df_openflight_routes['source-airport-id'].astype(str)
df_openflight_routes['dest-airport-id'] = df_openflight_routes['dest-airport-id'].astype(str)

print(df_openflight_routes)
print(df_openflight_routes.dtypes)


      airline-code airline-id source-airport-code source-airport-id dest-airport-code dest-airport-id codeshare  stops equipment
0               2B        410                 ASF              2966               KZN            2990       NaN      0       CR2
1               2B        410                 ASF              2966               MRV            2962       NaN      0       CR2
2               2B        410                 CEK              2968               KZN            2990       NaN      0       CR2
3               2B        410                 CEK              2968               OVB            4078       NaN      0       CR2
4               2B        410                 DME              4029               KZN            2990       NaN      0       CR2
...            ...        ...                 ...               ...               ...             ...       ...    ...       ...
67657           ZL       4178                 WYA              6334               ADL            

In [13]:
df_provided_airports = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/provided/AirportMapping.csv',
    skipinitialspace='true')
print(df_provided_airports)

                City      Country Code
0            Aalborg      Denmark  AAL
1           Aalesund       Norway  AES
2             Aarhus      Denmark  AAR
3     Abbotsford, BC       Canada  YXX
4     Abbotsford, BC       Canada  YXX
...              ...          ...  ...
1574        Zanzibar     Tanzania  ZNZ
1575        Zaragoza        Spain  ZAZ
1576       Zhengzhou        China  CGO
1577        Zhoushan        China  HSN
1578          Zurich  Switzerland  ZRH

[1579 rows x 3 columns]


In [190]:
# read provided route data
df_provided_routes = pd.read_csv(
    'https://raw.githubusercontent.com/stevecoffee/dasc5300/main/Project2/data/provided/routes_v3.txt',
    names=['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment'],
    skipinitialspace='true', header=0, na_values='\\N')
df_provided_routes = df_provided_routes.drop('codeshare', axis=1)
df_provided_routes

,airline-code,airline-id,source-airport-code,source-airport-id,dest-airport-code,dest-airport-id,stops,equipment
0,AL242,AL242,ASF,2966.0,KZN,2990.0,0,CR2
1,AL242,AL242,ASF,2966.0,MRV,2962.0,0,CR2
2,AL242,AL242,CEK,2968.0,KZN,2990.0,0,CR2
3,AL242,AL242,CEK,2968.0,OVB,4078.0,0,CR2
4,AL242,AL242,DME,4029.0,KZN,2990.0,0,CR2
...,...,...,...,...,...,...,...,...
67657,AL156,AL156,WYA,6334.0,ADL,3341.0,0,SF3
67658,AL530,AL530,DME,4029.0,FRU,2912.0,0,734
67659,AL530,AL530,FRU,2912.0,DME,4029.0,0,734
67660,AL530,AL530,FRU,2912.0,OSS,2913.0,0,734


In [191]:
# clean df_provided_routes
iataLookup = df_openflight_airports.set_index('iata')['airport-id'].to_dict()

# patch source airport id where possible
missing_source_mask = df_provided_routes['source-airport-id'].isna()
df_provided_routes.loc[missing_source_mask, 'source-airport-id'] = df_provided_routes.loc[missing_source_mask, 'source-airport-code'].map(iataLookup)

# patch dest airport id where possible
missing_dest_mask = df_provided_routes['dest-airport-id'].isna()
df_provided_routes.loc[missing_dest_mask, 'dest-airport-id'] = df_provided_routes.loc[missing_dest_mask, 'dest-airport-code'].map(iataLookup)
# print(df_provided_routes[mask])

missing_mask = df_provided_routes.isna().loc[:,('source-airport-id', 'dest-airport-id')].any(axis='columns')
df_provided_routes = df_provided_routes[missing_mask == False]

df_provided_routes


,airline-code,airline-id,source-airport-code,source-airport-id,dest-airport-code,dest-airport-id,stops,equipment
0,AL242,AL242,ASF,2966,KZN,2990,0,CR2
1,AL242,AL242,ASF,2966,MRV,2962,0,CR2
2,AL242,AL242,CEK,2968,KZN,2990,0,CR2
3,AL242,AL242,CEK,2968,OVB,4078,0,CR2
4,AL242,AL242,DME,4029,KZN,2990,0,CR2
...,...,...,...,...,...,...,...,...
67657,AL156,AL156,WYA,6334,ADL,3341,0,SF3
67658,AL530,AL530,DME,4029,FRU,2912,0,734
67659,AL530,AL530,FRU,2912,DME,4029,0,734
67660,AL530,AL530,FRU,2912,OSS,2913,0,734


In [145]:
# todo: Improve cleaning using redundant data.  Populate missing 
# print(df_provided_routes[df_provided_routes['source-airport-code'] == 'TGK'])
# print(df_provided_routes[df_provided_routes['dest-airport-code'] == 'TGK'])
# print(df_provided_airports[df_provided_airports['Code'] == 'TGK'])

iataLookup = df_openflight_airports.set_index('iata')['airport-id']

mask = df_provided_routes['source-airport-code'].isna()
print(df_provided_routes.loc[mask])
df_provided_routes.loc[mask, 'source-airport-id'] = df_provided_routes.loc[mask, 'source-airport-code'].apply(lambda x: iataLookup[x])
print(df_provided_routes.loc[mask])


In [160]:
key = 'SQS'
print(df_provided_routes[df_provided_routes['source-airport-code'] == key])
print(df_provided_routes[df_provided_routes['dest-airport-code'] == key])
print(df_provided_airports[df_provided_airports['Code'] == key])
print(df_openflight_airports[df_openflight_airports['iata'] == key])
print(df_openflight_airports[df_openflight_airports['icao'] == key])

     airline-code airline-id source-airport-code source-airport-id dest-airport-code  dest-airport-id codeshare  stops equipment
3776        AL220      AL220                 SQS               NaN               BCV           8227.0       NaN      0       CNC
3777        AL220      AL220                 SQS               NaN               BZE           1957.0       NaN      0       CNC
3778        AL220      AL220                 SQS               NaN               SPR           4279.0       NaN      0       CNC
3779        AL220      AL220                 SQS               NaN               TZA           6463.0       NaN      0       CNC
     airline-code airline-id source-airport-code source-airport-id dest-airport-code  dest-airport-id codeshare  stops equipment
3739        AL220      AL220                 BCV              8227               SQS              NaN       NaN      0       CNC
3751        AL220      AL220                 BZE              1957               SQS             

In [18]:
# create route graph
# https://igraph.org/python/api/latest/igraph.Graph.html#DataFrame
vertices = df_airports
directed = True
edges = df_routes
g = Graph.DataFrame(edges, directed, vertices)


# start=160
# size=1
# for i in range(start//size, len(vertices.index)//size-start):
#   print('rows',size*i, size*(i+1))
#   edges = df_routes[size*i:size*(i+1)]
#   try:
#     g = Graph.DataFrame(edges, directed, vertices)
#   except:
#     print(edges)
#     raise



NameError: ignored

In [8]:
print(g.vcount(), g.ecount())

7697 66305


In [9]:
# print(airportDict['3531'])
# print(df_airports[ df_airports['airport-id'] == '3531' ] )

# print(airportDict['7167'])
# print(df_airports[ df_airports['airport-id'] == '7167' ] )
# print(df_airports[ df_airports['airport-id'].str.contains('167') ] )
test = df_routes['source-airport-id'].isin(df_airports['airport-id'])
print(df_routes[ test==False ])

# print(df_airports[ df_airports['airport-id'] == '14106' ] )

Empty DataFrame
Columns: [source-airport-id, dest-airport-id, airline-code, airline-id, source-airport-code, dest-airport-code, codeshare, stops, equipment]
Index: []


In [10]:
# load anonymized airline route data set (to be provided)
raw_column_names = ['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment']
df_anon_routes = pd.read_csv(
    'https://raw.githubusercontent.com/jpatokal/openflights/master/data/countries.dat',
    names=['airline-code', 'airline-id', 'source-airport-code', 'source-airport-id', 'dest-airport-code', 'dest-airport-id', 'codeshare', 'stops', 'equipment'],
    skipinitialspace='true', header=0)

FileNotFoundError: ignored

In [ ]:
# separate airline assigned to me

In [ ]:
# Part 1a
# Construct a graph (or a graph data structure) for the airline being analyzed by you. With
# airport code as the vertex, draw an undirected edge between the two airports in each
# line of the input. Since many Python packages/algorithms only accept integers as vertex,
# you may have to map the 3-letter airport code into an integer. You can use a dictionary
# or something else for this purpose.

In [ ]:
# Part 1b
# Using the given method, generate a few characteristics for this airline route graph. This
# will help understand some of the characteristics of the data set you are analyzing.
# Interpret them from an airlines operation perspective.

In [ ]:
# Part 1c
# Find the top 3 to 5 nodes (or airports; top k, in general) from where there are more
# number of flights than other nodes for that airline. These are termed hubs by an airline.
# You can use a node centrality detection algorithm/package for this

In [ ]:
# Part 1d
# Once you have the top k hubs, try to identify the airline that you are working on. You
# can use any data that is available on the Internet for this purpose. We will give you a file
# that maps airport code to airport names and cities.

In [ ]:
# Part 1e
# Once you have the airline identified, verify that with us. In this step of the analysis,
# your goal is to identify or predict the next hub for airline expansion. For this, you can
# use demographic and other information (e.g., regional) that is available on the web.
# The purpose of this step is for you to develop a feel for the analysis beyond data
# usage. We will grade you based on the ideas you use/come up with for this analysis. The
# actual airport identified as the next hub is secondary. If it matches with the actual one,
# that is great. You need to argue why your process and the publicly available data you
# have used are appropriate.